In [ ]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

In [ ]:
#Install Packages
!pip install llama-index==0.5.6
!pip install langchain==0.0.148
!pip install PyPDF2
# !pip install streamlit
!pip install openai
# !pip install top2vec
!pip install pdfplumber

In [ ]:
!git clone https://github.com/MedX1736/QuizGPT.git

fatal: destination path 'QuizGPT' already exists and is not an empty directory.


## **Data Extraction From Pdf Files**

In [ ]:
import pdfplumber
import os
try :   
  os.mkdir("context")
  files = os.listdir("pdfData")
except :
  print("Data Not Available")

# creating a pdf file object
pdf = pdfplumber.open('QuizGPT/pdfData/chap9.pdf')

topics =[]
context= ""

# Extract and concatenate each page's content
for page in pdf.pages:
    # creating a page object
    # extracting text  and Titles from pdf
    context += page.extract_text()
    topics.append(page.filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"]).extract_text())
with open("context/text1.txt", 'w', encoding='utf-8') as output_file:
      output_file.write(context)

Data Not Available


# **Topics Extraction**

In [ ]:
import re
def clean_topics(topics):
  result = []
  for text in topics :
    splitted = text.split("\n")
    for topic in splitted :
      x = re.sub(r'^[\d\W_]+\s*', '', topic)
      if x != "" :
        result.append(x)
  return result

In [ ]:
topics = clean_topics(topics)

In [ ]:
from top2vec import Top2Vec
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
model = Top2Vec(documents=text, speed='learn',workers=multiprocessing.cpu_count())

In [ ]:
topics_words, word_scores, topic_nums = model.get_topics()
print(topics_words) # you can directly do model.topic_words[3] too
print(word_scores)

In [ ]:
topics

['Programmation Logique',
 'Introduction',
 'Rappel sur logique du premier ordre',
 "Définition d'un terme",
 "Définition d'un atome",
 "Définition d'une formule (énoncé)",
 'Conséquence logique',
 'Démonstration automatique de théorèmes',
 'Clauses',
 'Substitution',
 'Unificateur',
 'Unification de 2 termes : Couple de désaccord',
 'D(t1, t2)',
 'Unification de 2 termes : algorithme',
 'Système formel de la résolution',
 'Détermination des résolvantes : ( Explication )',
 'Algorithme général de résolution (Stratégie par saturation)',
 'Quelques cas particuliers de la résolution',
 'Variante de la résolution pour les clauses de Horn',
 'Algorithme de la résolution descendante :',
 "Signaler 'échec'",
 "Signaler 'Succès'",
 'Le langage Prolog',
 'Objets manipulés en Prolog',
 'Fonctionnement du démonstrateur automatique de Prolog',
 'Exemple de programme Prolog simple :',
 'Les listes en Prolog',
 'Les objet structurés']

In [ ]:
len(topics)

28

# **Constructing Index**

In [ ]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    max_input_size = 4096
    num_outputs = 2000
    max_chunk_overlap = 20
    chunk_size_limit = 600 

    # define prompt helpera
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs,language='fr'))
    documents = SimpleDirectoryReader(directory_path).load_data()
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
    index.save_to_disk('index.json')

    return index

In [ ]:
index  = construct_index("context")

                    language was transfered to model_kwargs.
                    Please confirm that language is what you intended.


In [ ]:
def generate_question():
  questions = []
  index = GPTSimpleVectorIndex.load_from_disk('index.json')
  for topic in topics :
    query = "Poser une Question sur "+ topic + "Et Donner la sous la forme Question : ... Reponse : ..."
    quiz = index.query(query)
    questions.append(quiz)
  return questions

In [ ]:
def generate_question_choix_multp():
  questions = []
  index = GPTSimpleVectorIndex.load_from_disk('index.json')
  for topic in topics :
    query = "Poser Une Question de choix multiple sur "+ topic + "Et Donner la reponse, sous la forme Question : ...  Reponse : ..."
    quiz = index.query(query)
    questions.append(quiz)
  return questions

In [ ]:
questions =  generate_question()

In [ ]:
quizs = generate_question_choix_multp()

In [ ]:
with open("questions/questions.txt", "w") as file:
    for question in questions:
        file.write(question.response + "\n")
with open("questions/quizs.txt", "w") as file:
    for quiz in quizs:
        file.write(quiz.response + "\n")

# Streamlit App

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
import streamlit as st

st.title('Streamlit on Colab')
st.write('Hello, world!')

2023-05-14 13:08:47.361 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(port='8501')
public_url

PyngrokNgrokHTTPError: ignored

In [ ]:
!streamlit run --server.port 8501 your_streamlit_app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: your_streamlit_app.py
